he utilizado print para ver lo que hacia cada linea de codigo. Previamente he leido la documentacion de la pagina de pandas, de scipy y de python en general para entender brevemente el funcionamiento. Le comento en el programa lo que entiendo de cada linea con sus funciones. he limpiado con restart and clear output para que no se muestren todas las salidas. algunos print los he dejado y otros los he quitado
ademas le anado como calcular la entropia segun he visto por internet

In [ ]:
#Practica 1: considero que esta bastante correcta

In [2]:
import numpy as np
from scipy.stats import entropy
import pandas as pd
from sklearn.datasets import make_blobs
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [3]:
#df = pd.read_csv("ejemplo.csv",names=['outlook','temperature','humidity','windy','play'])
df = pd.read_csv("weather.csv")

In [4]:
print(df)

     outlook temperature humidity  windy play
0      sunny         hot     high  False   no
1      sunny         hot     high   True   no
2   overcast         hot     high  False  yes
3      rainy        mild     high  False  yes
4      rainy        cool   normal  False  yes
5      rainy        cool   normal   True   no
6   overcast        cool   normal   True  yes
7      sunny        mild     high  False   no
8      sunny        cool   normal  False  yes
9      rainy        mild   normal  False  yes
10     sunny        mild   normal   True  yes
11  overcast        mild     high   True  yes
12  overcast         hot   normal  False  yes
13     rainy        mild     high   True   no


In [5]:
outlook = ['sunny','overcast','rainy']
temperature = ['hot','mild,','cold']
humidity = ['high','normal']
windy=['True','False']
play=['no','yes']

In [6]:
H = entropy(df.iloc[:, -1].value_counts(normalize=True), base=2)
#la funcion entropia de scipy.stats recibe en primer lugar la tabla con los datos. con iloc de :,-1 seleccionamos 
#la ultima columna, contando las apariciones que hay y normalize nos da en proporcion es decir en decimal
# base 2 es el logaritmo de base que utiliza

In [7]:
#df.iloc[:, -1].value_counts(normalize=True)
#porque 5 'noes' entre 14 son 0,35 de proporcion
#devuelve esto
#yes    0.642857
#no     0.357143

In [8]:
print(H)

0.940285958670631


In [9]:
G = []
l = len(df)
# utiliza len que es para obtener la longitud de la tabla
#df.columns[:-1] obtiene 'outlook', 'temperature', 'humidity', 'windy']
for atributo in df.columns[:-1]:
    G.append(H)
    # va anadiendo cada entropia que calcula. aqui anado la primera
    for v in df[atributo].unique():  
        #para atributo outlook encuentra sunny,overcast y rainy, ultimo false o true
        
        table = df.loc[df[atributo] == v]
        #0    sunny         hot     high  False   no
        #1    sunny         hot     high   True   no
        #7    sunny        mild     high  False   no
        #8    sunny        cool   normal  False  yes
        #10   sunny        mild   normal   True  yes
    
        #hace una minitabla por ejemplo con todas las filas que tienen sunny en el outlook
        #aqui utiliza loc para seleccionar filas en base a la condicion de que el atributo de la fila
        # sea igual al seleccionado en el bucle
        G[-1] -= (len(table)/l)*entropy(table.iloc[:, -1].value_counts(normalize=True), base=2)
        #longitud de la minitabla / entre la longitud de la tabla total * la entropia calcula de la minitabla
        # guardo la entropia al vector
        #formula: -p log2 (p)
H, G

(0.940285958670631,
 [0.246749819774439,
  0.029222565658954758,
  0.15183550136234159,
  0.04812703040826943])

In [10]:
Gi = []
G_Gini = 1-sum(df.iloc[:,-1].value_counts(normalize=True)**2)
#df.iloc[:,-1].value_counts(normalize=True) nos da yes:0,64 y no 0,35 y se hace el cuadrado
#hacemos la proporcion como en la entropia con el iloc y normalize, y la elevamos al cuadrado por la formula.
#eso lo restamos de 1 por la formula del indice de gini
#formula : 1-sumas(p²)
# esto es similar a los anteriores calculos de la entropia, cambiando solo la formula para el indice de gini
l = len(df)
#df.columns[:-1] obtiene 'outlook', 'temperature', 'humidity', 'windy']
for atributo in df.columns[:-1]:
    Gi.append(G_Gini)
     # va anadiendo cada gini que calcula. aqui anado la primera
    for v in df[atributo].unique():
        #para atributo outlook encuentra sunny,overcast y rainy, ultimo false o true
        table = df.loc[df[atributo] == v]
         #0    sunny         hot     high  False   no
        #1    sunny         hot     high   True   no
        #7    sunny        mild     high  False   no
        #8    sunny        cool   normal  False  yes
        #10   sunny        mild   normal   True  yes
        Gi[-1] -= (len(table)/l)*(1-sum(table.iloc[:,-1].value_counts(normalize=True)**2))
          #longitud de la minitabla / entre la longitud de la tabla total * gini calculada de la minitabla
G_Gini, Gi

(0.4591836734693877,
 [0.11632653061224485,
  0.018707482993197258,
  0.09183673469387743,
  0.030612244897959162])

In [49]:
def ent(data):
    #recibe minitabla por ejemplo con los que tienen hot su valor de play
    #3     yes
#4     yes
#5      no
#9     yes
#13     no
    prob1 = pd.value_counts(data) / len(data)
#yes    0.6 los otros son 3/5
#no     0.4
    #saca las proporciones para overcast por ejemplo de si y no
    return sum(np.log2(prob1) * prob1 * (-1))
    #formula de la entropia

In [52]:
def gain(data,str1,str2):
    e1 = data.groupby(str1).apply(lambda x:ent(x[str2]))
    #obtiene la cantidad de informacion de cada valor diferente de la columna, 
    #dos iguales y dos dif=1, todos iguales=0
    p1 = pd.value_counts(data[str1]) / len(data[str1])
    #mild    0.428571 6/14
    #hot     0.285714
    #cool    0.285714
    #calcula la probabilidad de cada valor diferente de la columna
    e2 = sum(e1 * p1)
    
    #formula de la entropia
    #formula ganacia: entropia padre -hijos
    return ent(data[str2]) - e2
    #restamos la entropia calculada de la de la columna de referencia

In [53]:
gain(df, 'outlook', 'play')

outlook
overcast    0.000000
rainy       0.970951
sunny       0.970951
dtype: float64


0.24674981977443933

In [54]:
gain(df, 'temperature', 'play')

temperature
cool    0.811278
hot     1.000000
mild    0.918296
dtype: float64


0.02922256565895487

In [15]:
gain(df, 'humidity', 'play')

0.15183550136234159

In [16]:
gain(df, 'windy', 'play')

0.04812703040826949

yes    0.642857
no     0.357143
Name: play, dtype: float64


mild    0.428571
hot     0.285714
cool    0.285714
Name: temperature, dtype: float64
